In [1]:
import numpy as np
from itertools import product
import ast

In [250]:
geometry = np.loadtxt("geometry_test.txt", dtype = str)

In [251]:
# Creating an input file for gaussian calculation
# nproc - to specify the number of processos to be used in calculation, mem - to specify the memory to be used in calculation
# file_name = name by which the file will be saved
# geom - Geometry to be used in calculation
# keywords - keywords to be specified to gaussian. A string or a list of word is accepted
# title - title to be used in file
# oldchk = False, an old checkpoint name needs to be indicated, oldchk_file = Name of the old checkpoint file
# chk = False, put True if a chk with the same name as filename needs to be saved, chk_name = Name of the checkpoint file to be used instead
# charge_multiplicity = (0, 1) by default, if specified, another charge and multiplicity will be used
# basis set = False, if smth else, need to be indicated another basis set in form ["Atom1, Atom2, ...", "Basis Set Used"]. Both values are single strings
# wfx = False, A wfx file name needs to be specified
# Field = False, if smth else, the directions of field need to be specified
#

basis_set = {}


def create_gaussian_file(file_name, keywords, nproc=False, mem=False, title="Job Name", oldchk=False, oldchk_file=None, chk=False, chk_name=False,
                         charge_multiplicity=(0, 1), geom=False, basis_set=False, wfx=False, Field=False):

    file_gaussian = open(file_name, "w")  # Opening the file

    if nproc:  # Adding the line of nprocessors if the number of processors is specified
        file_gaussian.write("%nproc=" + str(nproc) + "\n")

    if mem:  # Adding the line with memory
        file_gaussian.write("%mem=" + mem + "\n")

    if oldchk:  # Checking if and old checkpoint is to be used in the calculation
        if oldchk_file == None:
            raise TypeError(
                "Old checkpoint file is not specified. Working filename is: " + file_name)
        # Adding the line of an old chk to the gaussian input
        file_gaussian.write("%oldchk=" + oldchk_file + "\n")

    if chk:  # Checking if the checkpoint needs to be created
        if chk_name:  # Case when name of chk is different from file name
            file_gaussian.write("%chk=" + chk_name + "\n")
        else:  # Chk point is the same as file name
            file_gaussian.write(
                "%chk=" + file_name[:-4].split("/")[-1] + ".chk\n")

    # Adding the keywords to the file

    if type(keywords) == list:  # Checking if keywords are specified as a list
        new_line = " ".join(keywords)
        file_gaussian.write(new_line + '\n\n')
    elif type(keywords) == str:  # Checking if keywords are specified as a string
        file_gaussian.write(keywords + "\n\n")
    else:
        # An error for the case of wrong keyword list
        raise TypeError(
            "keywords need to be a list of keywords or a string. Working filename is: " + file_name)

    file_gaussian.write(title + "\n\n")  # Adding the title
    # Adding charge and multiplicity. By default 0 1
    file_gaussian.write(" ".join(str(x) for x in charge_multiplicity) + "\n")
    if geom is not False:
        for i in range(geom.shape[0]):  # Writting the geometry
            file_gaussian.write(" ".join(geom[i]) + "\n")
        file_gaussian.write("\n")  # Adding a blanc line at the end of file
    else:
        file_gaussian.write("\n")

    if Field is not False:  # Adding lines corresponding to the Field
        file_gaussian.write(" ".join([str(x) for x in Field]) + "\n\n")

    if basis_set:  # Adding the lines corresponding to the basis set
        if type(basis_set[0]) == list:  # Checking if the atoms were saved as a list
            file_gaussian.write(" ".join(basis_set[0]) + " 0" + "\n")
        elif type(basis_set[0]) == str:
            file_gaussian.write(basis_set[0] + " 0" + "\n")
        # Writting the basis set in the file
        file_gaussian.write(basis_set[1] + "\n" + "****" + "\n\n")
    if wfx:  # write wfx file
        file_gaussian.write(wfx + "\n\n")
    file_gaussian.close()  # Closing the file


In [252]:
#Creating a matrix with values of how each element in the electric field changes
#ndim - dimension of matrix 
#type_space - "linear", "log" or "step". Linear - n values on the range of start to finish 
#log - logarithmicaly n spaced values on the range start to finish 
#step - values placed with a specified step from start to finish, finish is not included. 
#all the same = False, change to a [start, finish, step] for the case when for all the directions the change is the same 
#**kwargs submit the directions and how the field will be changing in the form of {"Direction" : [start, finish, step], "Direction2": [start, finish, step]}

def generate_input_energy_field_calculation(ndim, type_space, all_the_same = False, **kwargs):
    ndim_l = [3 for i in range(ndim)]                                                        #Getting the dimensions of the matrix
    matrix = np.zeros(ndim_l, dtype=object)                                                  #Creating a matrix. Having dtype = object is crucial here
    character_mapping = {"X" : "0", "x" : "0", "Y": "1", "y": "1", "Z" : "2", "z": "2"}      #Creating a map of letters into numbers(indexes)
    new_dict = {}                                                                            #To store the new data

    for old_key, value in kwargs.items():                                                    #Changing the letters into number
        for char, replacement in character_mapping.items():                                  #iterating over the map created earlier 
            old_key = old_key.replace(char, replacement)                                     #replacing character
        new_dict[old_key] = value
    
    if all_the_same is not False:                                                            #The case when we want all directions of electric field to have the same values
        if type_space == "linear":
            new_array = np.linspace(all_the_same[0], all_the_same[1], all_the_same[2])
        if type_space == "log":
            new_array = np.logspace(np.log10(all_the_same[0]), np.log10(all_the_same[1]), all_the_same[2])
        if type_space == "step":
            new_array = np.arange(all_the_same[0], all_the_same[1], all_the_same[2])
        for index, element in np.ndenumerate(matrix):     
            print(index, element)
            matrix[index] = new_array
        return matrix

    for key, value in new_dict.items():                                                      #Changing only the elements of the matrix that were specified in the kwargs
        new_key = [int(x) for x in key]                                                      #Making a list of integers from the str
        if type_space == "linear":
            new_array = np.linspace(value[0], value[1], value[2])
        if type_space == "log":
            new_array = np.logspace(np.log10(value[0]), np.log10(value[1]), value[2])
        if type_space == "step":
            new_array = np.arange(value[0], value[1] + 10**(-10), value[2])
        matrix[tuple(new_key)] = new_array                                           #Assigning the new values 
    return matrix
        

In [253]:
from itertools import product

def generate_gaussian_field_calculation(matrix, file_name, keywords, nproc=False, mem=False, title="Job Name", oldchk=False, oldchk_file=None, chk=False, chk_name=False,
                         charge_multiplicity=(0, 1), geom=False, basis_set=False, wfx=False, Field=False):
    
    def map_number_to_direction(j):
        character_mapping = {"0" : "X", "1": "Y", "2" : "Z"}
        j = str(j)
        new_str = ""
        for char in j:
            if char in character_mapping:
                new_str += character_mapping[char]
            else: new_str += char
        return new_str

        

    matrix = np.reshape(matrix, -1)

    for i in range(len(matrix)):
        if not isinstance(matrix[i], np.ndarray):
            matrix[i] = [matrix[i]]

    for i, Field in enumerate(product(*matrix)):
        A = []
        for j in range(len(Field)):
            A.append([map_number_to_direction(j), Field[j]])
        add_name_field = ["_".join([str(x1) for x1 in x]) for x in A]
        add_name_field = "_".join(add_name_field) 
        create_gaussian_file(file_name = file_name[:-4] + "_" + str(i) + "_" + add_name_field + ".inp", keywords = keywords, nproc=nproc, mem=mem, title=title, oldchk=oldchk, oldchk_file=oldchk_file, chk=chk, chk_name=chk_name,
                         charge_multiplicity=charge_multiplicity, geom=geom, basis_set=basis_set, wfx=wfx, Field=Field)
    return 


In [254]:
A = generate_input_energy_field_calculation(1, "linear", **{"X" : [10, 100, 5], "Y" : [10, 100, 5], "Z" : [1, 5, 5] })
name = "test_new_test.inp"
kw = "STO-3G"
generate_gaussian_field_calculation(A, file_name= name, keywords= kw, geom = geometry)

In [255]:
class Gaussian_File:
    def __init__(self, file_name = "name.inp", keywords = "", nproc=False, mem=False, title="Job Name", oldchk=False, oldchk_file=None, chk=False, chk_name=False,
                         charge_multiplicity=(0, 1), geom=False, basis_set=False, wfx=False, Field=False):
        self.file_name = file_name
        self.keywords = keywords
        self.nproc = nproc
        self.mem = mem 
        self.title = title 
        self.oldchk = oldchk
        self.oldchk_file = oldchk_file
        self.chk = chk
        self.chk_name = chk_name
        self.charge_multiplicity = charge_multiplicity
        self.geom = False 
        self.basis_set_gaussian = basis_set
        self.wfx = wfx
        self.Field = Field


In [ ]:
import ast
def start():

    File_Info = Gaussian_File()

    File_Info.file_name = input("1. Introduce the name of the file of your calculation. \n Including the path. If not, the files will be saved in the directory were this file is.\nExample /Users/User/Documents")
    #----------------------------------------------------------------------------------------------------------------------------------
    a = [] 
    i = 1 
    while True:
        keyword = input("2. Introduce the keywords. Each line is a new line for the keyword. Introduce 0 to finish the introduction of kw")
        if keyword == "0":
            break
        a.append(keyword)
        print("\n".join(a))
        print("Cycle number", i)
        print("-----------------")
        i += 1
    File_Info.keywords = "\n".join(a)
    #-------------------------------------------------------------------------------------------------------------------------------
    try:
        nproc = int(input("3. Introduce the number of processors to be used during the calculation. \n Introduce 0 if you do not want to specify the number of processors."))
        if nproc != 0:
            File_Info.nproc = nproc
        else: File_Info.nproc = False
    except ValueError:
        print("Invalid input. Please enter and integer. 0 if you do not wish to specify the number of processors. Introduce False")
    mem = input("4. Introduce the amount of memory to be used during the calculation. Also indicate the amount. Example: 100MB. \n if you do not want to specify the memory, leave the space blank. (i.e., press Enter)")
    if not bool(mem):
        mem = False
    File_Info.mem = mem
    
    #----------------------------------------------------------------------------------------------------------------------------------
    a = [] 
    i = 1 
    while True:
        line = input("5. Introduce title\Comment for the Job to be used. Each line is a new line for the title. Introduce 0 to finish the introduction.")
        if line == "0":
            break
        a.append(line)
        print("\n".join(a))
        print("Cycle number", i)
        print("-----------------")
        i += 1
    File_Info.title = "\n".join(a)
    #-------------------------------------------------------------------------------------------------------------------------------

    oldchk = input("6. Do you want to use an old chk file? Yes/No")
    if oldchk[0].lower() == "y":
        oldchk = True
        File_Info.oldchk = oldchk
        File_Info.oldchk_file = input("Introduce the name for the old chk file, including the extension")
    else: oldchk = False

    
    chk = input("7. Do you want to save an chk file for this calculation? Yes/No")
    if chk[0].lower() == "y":
        File_Info.chk = True
        chk_name = input("Introduce the chk file name. Leaving it blanc will result in using the same name as for the log file")
        if not bool(chk_name):
            chk_name = False
        else: File_Info.chk_name = chk_name
    
    File_Info.charge_multiplicity = tuple(input("8. Intorduce charge and multiplicity of the system. \nTwo integers separated by space are to be introduced. Example: 0 1"))
    
    basis_set_atoms = input("9. Introduce atoms for which u will use basis set.\nAtoms are separated by spaces. Example: Li C N O \n Leave blank if you do not want to specify basis set for atoms")
    if bool(basis_set_atoms):
        basis_set_name = input("Introduce the name for the basis set to be used")
        File_Info.basis_set_gaussian = [basis_set_atoms, basis_set_name]
    else: File_Info.basis_set_gaussian = False

    wfx = input("10. Do you want to save an wfx file? Leave blank if you do not wish to print wfx")
    if not bool(wfx):
        wfx = False
    File_Info.wfx = wfx

    Energy = input("11. Do you want/have to specify a electric field for this calculation? \n Y/N")
    type_energy = int(input("How do you want your energy to vary? \n Possible choices: 1. Linearly, 2. Step, 3. Logarithmicaly. Introduce the corresponding integer."))
    if type_energy == 1:
        type_energy = "linear"
    elif type_energy == 2:
        type_energy = "step"
    elif type_energy == 3:
        type_energy == "log"
    else: type_energy = "linear"
    
    if Energy.lower() == "y":
        ndim = int(input("Introduce number of dimension for the energy calculation. n = 1 corresponds to only X, Y, and Z. n = 2 corresponds to XX, YY, etc."))
    
        dict_direct = ast.literal_eval("{" + input("Introduce the directions for which you would like to perform the calculations.\nFormat for this is \
                        \n'XX':[start, finish, step/number of points], 'XZ' : [start, finish, number of points], etc \n \
                        Respecting the format above is crucial. The direction should correspond with number of dimensions. 'X' for dimension 1, 'XX' for dimension 2, etc. \
                        \n ") + "}")
        field_matrix = generate_input_energy_field_calculation(ndim, type_energy, **dict_direct)
        generate_gaussian_field_calculation(field_matrix, File_Info.file_name, File_Info.keywords, nproc = File_Info.nproc, mem = File_Info.mem, title = File_Info.title, oldchk= File_Info.oldchk, oldchk_file=File_Info.oldchk_file, chk = File_Info.chk, chk_name=File_Info.chk_name, charge_multiplicity=File_Info.charge_multiplicity, geom = geometry, basis_set = File_Info.basis_set_gaussian, wfx = File_Info.wfx)
    else: create_gaussian_file(File_Info.file_name, File_Info.keywords, nproc = File_Info.nproc, mem = File_Info.mem, title = File_Info.title, oldchk= File_Info.oldchk, oldchk_file=File_Info.oldchk_file, chk = File_Info.chk, chk_name=File_Info.chk_name, charge_multiplicity=File_Info.charge_multiplicity, geom = geometry, basis_set = File_Info.basis_set_gaussian, wfx = File_Info.wfx)

In [ ]:
a = "%oldchk=wverv.fchk" 
if "Heh" in a:
    print("Yay")

In [ ]:
a = "wdvwv.inp"
a[-4:]

'.inp'

In [ ]:
a = [1, 2, 3, 4, 5]

for i in a[1:]:
    print(i)

2
3
4
5


In [ ]:
a = np.array(([1, 2, 3], [4, 5, 6], [7, 8 ,9]))
print(a)

[[1 2 3]
 [4 5 6]
 [7 8 9]]


In [283]:
import numpy as np

matrix = np.array([[1, [1, 2, 3], 3],
                   [4, 5, 6],
                   [7, 8, 9]])

mapping = {}

it = np.nditer(matrix, flags=["multi_index", "refs_ok"])
while not it.finished:
    index = it.multi_index  # Get the current multi-index
    number = it[0]  # Get the number at the current position
    mapping[str(number)] = index
    it.iternext()

# Print the mapping
for key, value in mapping.items():
    print(key, value)


1 (0, 0)
[1, 2, 3] (0, 1)
3 (0, 2)
4 (1, 0)
5 (1, 1)
6 (1, 2)
7 (2, 0)
8 (2, 1)
9 (2, 2)


/var/folders/ym/26b79b9x6wg59yqrctz2c40h0000gn/T/ipykernel_17375/498175774.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  matrix = np.array([[1, [1, 2, 3], 3],


In [291]:
it = np.nditer(matrix, flags=['multi_index', "refs_ok"])
for i in it:
    print(it[0])
    if isinstance(it[0], np.ndarray):
        #print(it[1])
        print("huh")

1
huh
[1, 2, 3]
huh
3
huh
4
huh
5
huh
6
huh
7
huh
8
huh
9
huh


In [265]:
it.multi_index

ValueError: Iterator is past the end

In [292]:
def create_mapping_from_n_dim_to_one_dim(matrix_for_mapping):
        mapping = {}

        it = np.nditer(matrix_for_mapping, flags=["multi_index", "refs_ok"])
        while not it.finished:
            index = it.multi_index  # Get the current multi-index
            number = it[0]  # Get the number at the current position
            mapping[str(number)] = index
            it.iternext()
        print(mapping)
        return mapping

create_mapping_from_n_dim_to_one_dim(matrix)

{'1': (0, 0), '[1, 2, 3]': (0, 1), '3': (0, 2), '4': (1, 0), '5': (1, 1), '6': (1, 2), '7': (2, 0), '8': (2, 1), '9': (2, 2)}


{'1': (0, 0),
 '[1, 2, 3]': (0, 1),
 '3': (0, 2),
 '4': (1, 0),
 '5': (1, 1),
 '6': (1, 2),
 '7': (2, 0),
 '8': (2, 1),
 '9': (2, 2)}

In [4]:
np.logspace(1, 3, 3)

array([  10.,  100., 1000.])

In [1]:
a = [1, 2, 3, 4, 5, 6, 7, 8]
a[1:4]

[2, 3, 4]

In [38]:
def vary_e_field_in_certain_direction(c1, c2, c3, var_range, type_coordinates = "cartesian", type_space = "linear"):
    """
    Function to vary the electric field in a certain direction.
    c1, c2, c3 - coordinates of the point in the space, where the electric field is to be varied. 
    format of the coordinates is (x, y, z) for catesian, (r, theta, phi) for spherical, (r, phi, h) for cylindrical
    Physics convetion is used. 
    The use of (r, theta, phi) denotes radial distance, inclination (or elevation), and azimuth, respectively.
    var_range - range of the variation.
    type_coordinates - "cartesian", "spherical", "cylindrical". Default is cartesian.
    type_space - "linear", "log", "step". Default is linear.

    The function will convert the coordinates in the sphericall coordinates. 
    Then it will vary the radial distance which represents the intensity of the electric field.
    """
    def convert_spherical_to_cartesian_coordinates(r, theta, phi):
        """
        Convert spherical coordinates to cartesian coordinates.
        The use of (r, theta, phi) denotes radial distance, inclination (or elevation), and azimuth, respectively.
        θ the angle measured away from the +Z axis 
        As φ has a range of 360° 
        θ has a range of 180°, running from 0° to 180°
        """
        x = r * np.sin(theta) * np.cos(phi)
        y = r * np.sin(theta) * np.sin(phi)
        z = r * np.cos(theta)
        return x, y, z
    
    
    def convert_cartesian_to_spherical_coordinates(x, y, z):
        """
        Convert cartesian coordinates to spherical coordinates.
        The use of (x, y, z) denotes the cartesian coordinates.
        """
        r = np.sqrt(x**2 + y**2 + z**2)
        theta = np.arccos(z / r)
        if int(x) == 0:
            phi = np.arctan(np.inf)
        else: phi = np.arctan(y / x)
        return r, theta, phi
    
    def convert_cylindrical_to_spherical_coordinates(r, phi, h):
        """
        Convert cylindrical coordinates to spherical coordinates.
        The use of (r, phi, z) denotes radial distance, azimuth angle, and height, respectively.
        """
        ro = np.sqrt(r**2 + h**2)   
        if int(h) == 0:
            theta = np.arctan(np.inf)
        else: theta = np.arctan(r / h)   
        return ro, theta, phi

    def return_x_y_z(x, y, z):                                               #Function to return the same coordinates
        return x, y, z

    if type_space == "linear":                                               #Creating the space for the variation
        space = np.linspace(var_range[0], var_range[1], var_range[2])
    elif type_space == "step":
        space = np.arange(var_range[0], var_range[1] + var_range[2], var_range[2])
    elif type_space == "log":
        space = np.logspace(np.log10(var_range[0]), np.log10(var_range[1]), var_range[2])
    else: space = np.linspace(var_range[0], var_range[1], var_range[2])

    #Mapping the function to the corresponding conversion of coordinates
    map_function = {"cartesian" : convert_cartesian_to_spherical_coordinates, "spherical" : return_x_y_z, "cylindrical" : convert_cylindrical_to_spherical_coordinates}
    f = map_function[type_coordinates]
    return_vector = []
    r, theta, phi = f(c1, c2, c3)                         #Converting the coordinates to spherical
    for i in space:                                       #Obtaining the new values for electric field
        r = i                                             #Length of vector is changed
        x, y, z = convert_spherical_to_cartesian_coordinates(r, theta, phi)        #Obtaining the new cartesian coordinates to be used in the Gaussian input file.
        x, y, z = np.round((x, y, z), 10)                                          #Rounding the values to 10 decimal places
        return_vector.append((x, y, z))                                            #Appending the new values to the list
    return return_vector

    


In [39]:
vary_e_field_in_certain_direction(0, 4, 1, var_range=[-4, 4, 1], type_coordinates="cartesian", type_space="step")

square of the x, y, z is: 4.0000000000071205
x, y, z values are -0.0, -3.8805700006, -0.9701425001
--------------------------------------------------
square of the x, y, z is: 2.9999999999628963
x, y, z values are -0.0, -2.9104275004, -0.7276068751
--------------------------------------------------
square of the x, y, z is: 2.000000000015687
x, y, z values are -0.0, -1.9402850003, -0.4850712501
--------------------------------------------------
square of the x, y, z is: 0.9999999999472096
x, y, z values are -0.0, -0.9701425001, -0.242535625
--------------------------------------------------
square of the x, y, z is: 0.0
x, y, z values are 0.0, 0.0, 0.0
--------------------------------------------------
square of the x, y, z is: 0.9999999999472096
x, y, z values are 0.0, 0.9701425001, 0.242535625
--------------------------------------------------
square of the x, y, z is: 2.000000000015687
x, y, z values are 0.0, 1.9402850003, 0.4850712501
-----------------------------------------------

In [33]:
np.sin(np.pi/2)

1.0

In [2]:
def vary_e_field_in_certain_direction(c1, c2, c3, var_range, type_coordinates = "cartesian", type_space = "linear"):
    """
    Function to vary the electric field in a certain direction.
    c1, c2, c3 - coordinates of the point in the space, where the electric field is to be varied. 
    format of the coordinates is (x, y, z) for catesian, (r, theta, phi) for spherical, (r, phi, h) for cylindrical
    Physics convetion is used. Values are to be given in degrees
    The use of (r, theta, phi) denotes radial distance, inclination (or elevation), and azimuth, respectively.
    var_range - range of the variation.
    type_coordinates - "cartesian", "spherical", "cylindrical". Default is cartesian.
    type_space - "linear", "log", "step". Default is linear.

    The function will convert the coordinates in the sphericall coordinates. 
    Then it will vary the radial distance which represents the intensity of the electric field.
    """
    def convert_spherical_to_cartesian_coordinates(r, theta, phi):
        """
        Convert spherical coordinates to cartesian coordinates.
        The use of (r, theta, phi) denotes radial distance, inclination (or elevation), and azimuth, respectively.
        θ the angle measured away from the +Z axis 
        As φ has a range of 360° 
        θ has a range of 180°, running from 0° to 180°
        """
        x = r * np.sin(theta) * np.cos(phi)
        y = r * np.sin(theta) * np.sin(phi)
        z = r * np.cos(theta)
        return x, y, z
    
    
    def convert_cartesian_to_spherical_coordinates(x, y, z):
        """
        Convert cartesian coordinates to spherical coordinates.
        The use of (x, y, z) denotes the cartesian coordinates.
        """
        r = np.sqrt(x**2 + y**2 + z**2)
        theta = np.arccos(z / r)
        if int(x) == 0:
            phi = np.arctan(np.inf)
        else: phi = np.arctan(y / x)
        return r, theta, phi
    
    def convert_cylindrical_to_spherical_coordinates(r, phi, h):
        """
        Convert cylindrical coordinates to spherical coordinates.
        The use of (r, phi, z) denotes radial distance, azimuth angle, and height, respectively.
        """
        ro = np.sqrt(r**2 + h**2)   
        if int(h) == 0:
            theta = np.arctan(np.inf)
        else: theta = np.arctan(r / h)   
        return ro, theta, phi

    def return_x_y_z(x, y, z):                                               #Function to return the same coordinates
        return x, y, z

    if type_space == "linear":                                               #Creating the space for the variation
        space = np.linspace(var_range[0], var_range[1], var_range[2])
    elif type_space == "step":
        space = np.arange(var_range[0], var_range[1] + var_range[2], var_range[2])
    elif type_space == "log":
        space = np.logspace(np.log10(var_range[0]), np.log10(var_range[1]), var_range[2])
    else: space = np.linspace(var_range[0], var_range[1], var_range[2])

    #Mapping the function to the corresponding conversion of coordinates
    map_function = {"cartesian" : convert_cartesian_to_spherical_coordinates, "spherical" : return_x_y_z, "cylindrical" : convert_cylindrical_to_spherical_coordinates}
    f = map_function[type_coordinates]
    return_vector = []
    if type_coordinates != "cartesian":
        c2 = np.radians(c2)                               #Converting from degrees to radians
        c3 = np.radians(c3)
    r, theta, phi = f(c1, c2, c3)                         #Converting the coordinates to spherical
    for i in space:                                       #Obtaining the new values for electric field
        r = i                                             #Length of vector is changed
        x, y, z = convert_spherical_to_cartesian_coordinates(r, theta, phi)        #Obtaining the new cartesian coordinates to be used in the Gaussian input file.
        x, y, z = np.round((x, y, z), 10)                                          #Rounding the values to 10 decimal places
        return_vector.append((x, y, z))                                            #Appending the new values to the list
    return return_vector

In [3]:
def generate_input_energy_field_calculation(ndim, type_space, all_the_same = False, **kwargs):
    ndim_l = [3 for i in range(ndim)]                                                        #Getting the dimensions of the matrix
    matrix = np.zeros(ndim_l, dtype=object)                                                  #Creating a matrix. Having dtype = object is crucial here
    character_mapping = {"X" : "0", "x" : "0", "Y": "1", "y": "1", "Z" : "2", "z": "2"}      #Creating a map of letters into numbers(indexes)
    new_dict = {}                                                                            #To store the new data

    for old_key, value in kwargs.items():                                                    #Changing the letters into number
        for char, replacement in character_mapping.items():                                  #iterating over the map created earlier 
            old_key = old_key.replace(char, replacement)                                     #replacing character
        new_dict[old_key] = value
    
    if all_the_same is not False:                                                            #The case when we want all directions of electric field to have the same values
        if type_space == "linear":
            new_array = np.linspace(all_the_same[0], all_the_same[1], all_the_same[2])
        if type_space == "log":
            new_array = np.logspace(np.log10(all_the_same[0]), np.log10(all_the_same[1]), all_the_same[2])
        if type_space == "step":
            new_array = np.arange(all_the_same[0], all_the_same[1], all_the_same[2])
        for index, element in np.ndenumerate(matrix):     
            print(index, element)
            matrix[index] = new_array
        return matrix

    for key, value in new_dict.items():                                                      #Changing only the elements of the matrix that were specified in the kwargs
        new_key = [int(x) for x in key]                                                      #Making a list of integers from the str
        if type_space == "linear":
            new_array = np.linspace(value[0], value[1], value[2])
        if type_space == "log":
            new_array = np.logspace(np.log10(value[0]), np.log10(value[1]), value[2])
        if type_space == "step":
            new_array = np.arange(value[0], value[1] + 10**(-10), value[2])
        matrix[tuple(new_key)] = new_array                                                   #Assigning the new values 
    return matrix

In [26]:
def create_mapping_from_n_dim_to_one_dim(matrix_for_mapping):                       
        mapping = {}                #Dictionary to save the mapping. 

        it = np.nditer(matrix_for_mapping, flags=["multi_index", "refs_ok"])              #Iteration over all values of matrix. 

        i = 0                                                                             #Index in the reshaped matrix. 
        while not it.finished:                                                            #Loop to iterate over all the matrix.
            index = it.multi_index                                                        #Get the index in the original matrix. 
            mapping[str(i)] = index                                                       #Creating a map of index in reshaped matrix as a key, and index in orginal matrix as its value. 
            it.iternext()                                                                 #Going to next value in the original matrix. 
            i += 1                                                                        #Going to next value in reshaped matrix. 
        return mapping 

In [27]:
def map_number_to_direction(j, map_1):
    letter_mapping = {"0" : "X", "1" : "Y", "2" :"Z"}                                 #Mapping of index into letters
    j = str(j)
    new_str = ""
    character_mapping = map_1                                                         #Having a map between the linear index and indexing in the original matrix. 
    j = "".join([str(x) for x in character_mapping[j]])                               #Obtaining the index in original matrix as a tuple, and transforming it into a string without spaces.
    for char in str(j):                                                               #A loop to transform the original indexing into letters.
        if char in letter_mapping:                                         
            new_str += letter_mapping[char]                                           #Using the map to get the letter for the char i in the name
        else: new_str += char                                                         #To avoid errors, if the char is not in the map, to return the character
    return new_str                                                                    #Returning the string


In [38]:
def read_input_for_electric_field_from_file_and_generate_files(path_to_file, extension = ".com"):
    """
    Function to generate Gaussian input files for the calculation of the electric field from a template file.
    Template will be specified in the manual.
    This funciton can generate files for the case when the electric field is varied in one specific direction or 
    it can generate a grid over a space. 
    This function is using the four following function:
    generate_input_energy_field_calculation(ndim, type_space, all_the_same = False, **kwargs)
    vary_e_field_in_certain_direction(c1, c2, c3, var_range, type_coordinates = "cartesian", type_space = "linear")
    create_mapping_from_n_dim_to_one_dim(matrix)
    map_number_to_direction(i, map_directions)
    """                                             
    type_electric_field_calculation = 0
    input = 0
    is_recording_file = True                                           #Boolean to indicate if we are recording the file
    is_recording_e_field = False                                       #Boolean to indicate if we are recording the electric field
    count = 0                                                          #To get the value where we will be inserting electric field
    lines = []                                                         #Saved lines of the file
    with open(path_to_file, "r") as file:                              
        for line in file:
            if line.strip() == "***Start_e_field***":                  #Looking for the start of the desired section so that we can get the input data
                is_recording_e_field = True                            #Boolean to let us read input data
                is_recording_file = False                              #Stop recording file lines to be in the final file
                line_to_introduce_e_field = count - 1
            if is_recording_file:                                      #Recording the lines of the file
                lines.append(line)
            if is_recording_e_field:                                   #Recording the input data
                if "#Corresponding" in line:                           #Read the type of calculation
                    type_electric_field_calculation = int(line.strip().split(" ")[-1])
                if "#Input" in line:                                   #Read input data
                    input = [x for x in line.strip().split(" ")[1:]]
            if line.strip() == "***Finish_e_field***":                 #Look for the finish of the block of reading the input data.
                is_recording_e_field = False                           #Stop recording input data
                is_recording_file = True                               #Continue recording the file
            count += 1                                                 #Counting the lines of the file
    print(type_electric_field_calculation)
    if type_electric_field_calculation == 2:                           #If recording the variation of electric field in one direction
        map_directions = {"0" : "X", "1" : "Y", "2" :"Z"}              #Dictionary to map the index of the direction into the letter
        #Generate the values of the electric field over the specified direction
        e_fields = vary_e_field_in_certain_direction(float(input[0]), float(input[1]), float(input[2]), var_range = [float(input[3]), float(input[4]), int(input[5])], type_coordinates = input[6], type_space = input[7])
        for field in e_fields:                                         #Looping to creating the files
            #Creating the name of the file. File name will have the following format:
            #Input_kw example_test_X-1.41e+00_Y-1.41e+00
            file_name = path_to_file[:-4] + "_" + "_".join(["".join((map_directions[str(i)], "+" + "{:.2e}".format(x) if x > 0 else "{:.2e}".format(x))) for i, x, in enumerate(field) if x!=0]) + extension
            with open (file_name, "w") as file:                        #Creating the file
                count = 0                                              #Counting lines of the file
                for line in lines:                                     #Writting line of the file
                    file.write(line)
                    if count == line_to_introduce_e_field:             #Add value of e_field in the specified line
                        file.write(" ".join([str(x) for x in field]) + "\n")
                    count += 1
    elif type_electric_field_calculation == 1:                         #Case of generating a grid over the space of the electric field. But can also be generated grids in n dimensions
        input[2] = True if input[2].lower() == "y" else False          #Having all the values the same
        input[3] = ast.literal_eval(input[3])                          #Reading the directions
        #Generating the matrix of how the electric field will vary. A position in the matrix corresponds to a direction.
        matrix = generate_input_energy_field_calculation(int(input[0]), input[1], input[2], **input[3])     
        #Creating a map of directions from n dimensions to one dimension
        #The resulting dictionary has the form {'0': (0, 0), '1': (0, 1), '2': (0, 2), ...}
        map_directions = create_mapping_from_n_dim_to_one_dim(matrix)
        matrix = np.reshape(matrix, -1)
        for i in range(len(matrix)):    #Make all non_list elements into a list. This is important for the product function.
            if not isinstance(matrix[i], np.ndarray):
                matrix[i] = [matrix[i]]
        for field in product(*matrix):  #Looping over all the possible combinations of the electric field. product function is used to get all the combinations.
            #File name will be of the following format: 
            #Input_kw example_test_X+1.41e+00_Y+1.41e+00
            file_name = path_to_file[:-4] + "_" + "_".join(["".join((map_number_to_direction(i, map_directions), "+" + "{:.2e}".format(x) if x > 0 else "{:.2e}".format(x))) for i, x, in enumerate(field) if x!=0]) + extension
            with open (file_name, "w") as file:                #Creating the files with the specified name and e field
                count = 0
                for line in lines:
                    file.write(line)                           #Writting the lines of the file
                    if count == line_to_introduce_e_field:     #Writting the e field in the specified line
                        file.write(" ".join([str(x) for x in field]) + "\n")
                    count += 1
    return type_electric_field_calculation, input

In [36]:
A = read_input_for_electric_field_from_file_and_generate_files("/Users/petrumilev/Documents/projects_python/File_for_proj_girona_donostia/Test_Input/Input_kw example_test.txt", extension=".txt")

line for e_field to be introduced 16
1


In [83]:
a = ("a", "b", "c")
"_".join(a)


'a_b_c'

In [113]:
# Example number
number = 0.000001234567

# Using format() method
print("{:.2e}".format(number))

# Using f-string (Python 3.6 and above)
print(f"{number:.2e}")


1.23e-06
1.23e-06


In [111]:
# Example number
number = 1234567.89

# Scientific notation with reduced decimals using format() method
formatted_number = "{:.2e}".format(number)
print(formatted_number)

# Scientific notation with reduced decimals using f-string (Python 3.6 and above)
formatted_number_fstring = f"{number:.2e}"
print(formatted_number_fstring)


1.23e+06
1.23e+06


In [122]:
a = "False"
bool(a)

True

In [45]:
added_kw = []
if added_kw:
    print("no")